In [16]:
import pandas as pd
from tqdm import tqdm
from rapidfuzz import fuzz, process
import numpy as np
import string
tqdm.pandas()

##This portion just surpresses warnings that do not affect the performance of the code.
import warnings
warnings.filterwarnings("ignore")

In [17]:

#This chunk contains pre-processing functions that format the data for both datasets


def prep_col(text, type='Name'):
    '''
    inputs: 
    text: a single string that is going to be processesd
    type: a string value that is either, 'Name' or 'Address' that dictates the exact type of processing

    outputs: 
    cleaned_text: A string that removes all selecd phrases and trailing spaces

    Descriptions: This funciton takes individual strings and regularizes them in terms of capitalization and spacing and reformats commmon phrases that appears in both
    Business Name and Address data
    '''
   #Make input text lower case, punctuationless, and string type
    text = str(text)
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))

    #Unique Handling for "Name" type strings
    if type == 'Name':
        #removes the following phrases in gen_char
        gen_char = {
            ' inc': '',
            ' llc': '',
            ' ltd': '',
            ' dba ': '',
            ' llp': ''
        }
        #Handles dba by removing the dba and replacing it with the text after it
        for sub, replacement in gen_char.items():
            if sub == ' dba ':
                text = text.split(sub, 1)[-1]
            else:
                text = text.replace(sub, replacement)
    #Unique Handling for Address type data
    elif type == 'Address':
        #Dictionary of common phrases and their preferred replacements
        replace_phrases = {
            ' plaza': 'plz',
            ' street': ' st',
            ' boulevard': ' blvd',
            ' avenue': ' ave',
            ' road': ' rd',
            ' lane': ' ln',
            ' circle': 'cir',
            ' suite': ' ste',
            ' floor':' fl',
            ' court':'ct',
            ' west': ' w',
            ' east': ' e',
            ' north': ' n',
            ' south': ' s',
            '#':'ste',
            'deleted':'',
            #Subaddress identifiers -> replacing with a generic pound sign for matching later on
            # Note: Does not affect original data
            ' ste': ' #', 
            ' apt':' #', 
            ' unit': ' #', 
            ' bldg': ' #', 
            ' fl':' #'
        }
        #Loop for addresses
        for phrase, replacement in replace_phrases.items():
            text = text.replace(phrase, replacement)
    #Final step that removes leading and trailing white space
    text = text.rstrip().lstrip()
    return text

def Pre_Zip(df, column_name):
    '''
    inputs: 
    df: a pandas dataframe containing the zipcode data we want to process
    column_name: the name of the column that contains the string data

    outputs: 
    df[column_name]: a new column of zipcode data that is appended to input dataframe

    Descriptions:
    This function simply regularizes the zip codes to snure that they are all of numeric type and 5 digits
    '''
    # Remove all non-numeric characters 
    df[column_name] = df[column_name].str.replace(r'[^0-9]', '', regex=True)
    
    # Replace empty/invalid strings
    df[column_name] = df[column_name].fillna('0').replace('', '0')
    
    # Truncate to 5 characters
    df[column_name] = df[column_name].str.slice(0, 5)
    
    # Integer type conversion
    df[column_name] = pd.to_numeric(df[column_name], errors='coerce').fillna(0).astype(int)
    
    return df[column_name]

def extract_add_num(input_string):
    ''' 
    inputs: 
    input string: A piece of string type data. Intended to be an address

    outputs: 
    numeric_substring: This is the leading number of an address
    remaining_string: This is the rest of the address 

    Description: This function simply splits addresses into their number and their remaining street addresses. 
    It is necessary for later functions.
    '''
    numeric_substring = ""  
    remaining_string = ""   
    
    #This loop searches for initial numbers and stops once a non-number is found
    for char in input_string:
        if char.isdigit():
            numeric_substring += char  
        else:
            remaining_string = input_string[len(numeric_substring):]  
            break  
    
    return numeric_substring, remaining_string

In [18]:
#Dataread for Government Data
Gpath=r"C:\Users\khat\OneDrive - PENNSYLVANIA COMPENSATION RATING BUREAU\Desktop\DNB_Data_Matching\LOCAL DNB_MATCH DATA\Generated\Joint_PADE_GOV_Data.csv"
GDF=pd.read_csv(Gpath)
#Dropping empty lead column
GDF=GDF.drop(GDF.columns[0], axis=1)
#Applying Business Name Cleaning
GDF['Adj_BN']=GDF['Adj_BN'].apply(lambda x: prep_col(x))
#Fixing null values in Address 2 column
GDF['Address 2']=GDF['Address 2'].fillna("")
#Appending Address 2 to Adress 1 to match with Optimus formatting in new adjusted address column
GDF['Adj_Add']= GDF['Address 1']+' ' + GDF['Address 2']
#Applying prep column function to entire adjusted address column
GDF['Adj_Add']=GDF['Adj_Add'].apply(lambda x: prep_col(x, 'Address'))
#Applying zip code cleaning
GDF['Zip']=GDF['Zip'].astype(str)
GDF['Zip']=Pre_Zip(GDF, 'Zip')
#Applying Address Splitting
GDF[['G_Add_#', 'G_Add_Name']] = GDF['Adj_Add'].apply(lambda x: pd.Series(extract_add_num(x)))
GDF['City']= GDF['City'].apply(lambda y: prep_col(y, 'City'))
#Shape check and Head Check for Government Data
print(GDF.shape)
GDF.head()

(2374999, 12)


,Business Name,Address 1,Address 2,City,State,Zip,Geocoded Location,Trade name,Adj_BN,Adj_Add,G_Add_#,G_Add_Name
0,"Greenacy, Llc",600 N 2nd Street,4th Floor,harrisburg,PA,17101,(-76.888068038 40.263499986),No Trade Name,greenacy,600 n 2nd st 4th #,600,n 2nd st 4th #
1,Alphamed Rx Inc.,502 W 7th Street,Suite 100,erie,PA,16502,(-80.095924977 42.12455901),No Trade Name,alphamed rx,502 w 7th st # 100,502,w 7th st # 100
2,Legam Llc,502 W 7th Street,Suite 100,erie,PA,16502,(-80.095924977 42.12455901),No Trade Name,legam,502 w 7th st # 100,502,w 7th st # 100
3,"Lycobirds, Llc",502 W 7th St,Ste 100,erie,PA,16502,(-80.095924977 42.12455901),No Trade Name,lycobirds,502 w 7th st # 100,502,w 7th st # 100
4,Cotoletta Llc,600 N 2nd Street,4th Floor,harrisburg,PA,17101,(-76.888068038 40.263499986),No Trade Name,cotoletta,600 n 2nd st 4th #,600,n 2nd st 4th #


In [19]:
#Dataread for Opti Data
#Im not commenting this code well because I think we will have to throw it out on Snowflake
OP_path=r"G:\SHEETS\Research\D&B\DataMatching\Businesses with multiple locations only_11_25\Single\ALL_Single_File.csv"
ODF=pd.read_csv(OP_path)
ODF=ODF[ODF['MATCH_SCORE']<.78]
ODF.drop(['MATCHED_BUSINESS_NAME', 'NAME_SCORE',
          'STREETADDRESS_DNB', 'ADDRESS_SCORE',
          'CITY_DNB', 'CITY_SCORE',
          'ZIP_DNB', 'ZIP_SCORE', 'MATCH_SCORE'], axis=1, inplace=True)
ODF['Cleaned_Name']=ODF['PRIMARYINSUREDNAME'].apply(lambda x: prep_col(x))
ODF['OC_Add']=ODF['STREETADDRESS_OPT'].apply(lambda x: prep_col(x, 'Address'))
ODF['ZIP_OPT']= ODF['ZIP_OPT'].astype(str)
ODF['ZIP_OPT']= Pre_Zip(ODF, 'ZIP_OPT')
ODF[['O_Add_#', 'O_Add_Name']] = ODF['OC_Add'].apply(lambda x: pd.Series(extract_add_num(x)))
ODF['CITY_OPT']= ODF['CITY_OPT'].apply(lambda y: prep_col(y, 'City'))
print(ODF.shape)
ODF.head()

(127266, 8)


,PRIMARYINSUREDNAME,STREETADDRESS_OPT,CITY_OPT,ZIP_OPT,Cleaned_Name,OC_Add,O_Add_#,O_Add_Name
1,HYDRITE CHEMICAL CO,208 WASHINGTON AVE,ellendale,19941,hydrite chemical co,208 washington ave,208,washington ave
2,SUNRISE VALLEY,CONSTRUCTION LLC 239 SPRINGVILLE RD,kinzers,17535,sunrise valley,construction llc 239 springville rd,,construction llc 239 springville rd
4,DIOCESAN CENTRAL COUNCIL OF GREENSBURG,70 N MT VERNON AVE,uniontown,15401,diocesan central council of greensburg,70 n mt vernon ave,70,n mt vernon ave
5,HOSPITAL HOUSEKEEPING SYSTEMS LLC,155 WILSON AVE,washington,15301,hospital housekeeping systems,155 wilson ave,155,wilson ave
7,ANYWHERE REAL ESTATE INC,3865 REED BLVD,murrysville,15668,anywhere real estate,3865 reed blvd,3865,reed blvd


In [20]:
#Creating new columns for each dataset that just contains the first 4 digits zip code
#This allows for the code to only check nearby businesses
ODF['APPROX_ZIP_OPT'] = ODF['ZIP_OPT'].astype(str).str[:4].astype(int)
GDF['APPROX_ZIP_GOV'] = GDF['Zip'].astype(str).str[:4].astype(int)

In [21]:
#These are the submatching functions that actually apply the matching for individual rows
#================================================
#Secondary Address handler:
def suite_cleaner(text1, text2):
    ''' 
    Inputs: 
    text1: A single string of address data from optimus data
    text2: A single string of address data from Gov data

    Ouputs: A single Boolean

    Description: This function matches secondary address numbers/codes such as suite, floor, unit etc. and returns a boolean if the
    addresses have matching secondary address numbers. True indicates either a match or an assymetry of data, 
    meaning one or both of the addresses doesnt have a secondary address. This boolean is used in an upcoming function
    ''' 
    #Remove all spaces from both input strings
    text1 = text1.replace(" ", "")
    text2 = text2.replace(" ", "")

    #Find the generic pound sign symbolizing a the prescence of a secondary address
    index1 = text1.find('#')
    index2 = text2.find('#')
    
    #If either are missing the pound sign, return True
    if index1 == -1 or index2 == -1:
        return True 
    #Log the secondary address numbers
    else:
        start1 = index1 + 1
        start2 = index2 + 1    
        text1= text1[start1:]
        text2= text2[start2:]

        #Check for secondary Address equality
        if text1 == text2:
            return True
        else:
            return False


#==========================================
#Address Fuzzy Matcher:
def a_fm(num1,rem1,num2, rem2):
    '''
    Input:
    num1: The leading address number from an address string from optimus
    num2: The leading address number from an address string from Gov data
    rem1: The second part of an address string from optimus
    rem2: The second part of an address string from Gov

    Output:
    final_score: a match score from 0 - 1 that indicates the match score

    Description: This function using the simple rapidfuzz ratio (normalized distance) on the rem1 and rem2 part of the addresses, but
    also applies custom penalties based on address number and secondary address numbers. This ensures more accurate address matches.

    
    '''
    #Removes leading and trailing spaces
    rem1=rem1.strip()
    rem2=rem2.strip()   
    
    #Checks if leading numbers match
    num_match = num1 == num2 
    #Fuzzy match address info
    fuzz_score = fuzz.ratio(rem1, rem2) / 100
    
    #Check if secondary address number matches
    sc=suite_cleaner(rem1, rem2)
    
    #Apply penalties to fuzzy match score
    #If suite numbers mismatch, apply a -.2 penalty
    if sc is False:
        fuzz_score-=.2

    #If leading numbers do not match: apply a -.35 penalty
    if not num_match:
        fuzz_score -= 0.35  
    
    #Safety Catch to stop negative scores from occuring, thus setting the floor at 0
    final_score = max(fuzz_score, 0) 
    
    return final_score
#===========================================
#Name fuzzy match
def n_fm(b1,b2):
    ''' 
    Inputs: 
    b1: a single string containg the business name from optimus
    b2: a single string containg the business name from the government data

    Outputs:
    nscore: A fuzzymatch score from 0-1

    Description: This function averages two distinct fuzzy matching scoring algorithms, Weighted ratio and standard ratio,

    '''
    Rscore=fuzz.ratio(b1,b2)/ 100
    Wscore=fuzz.WRatio(b1,b2)/100
    nscore=(Wscore + Rscore)/2
   
    return(nscore)

#========================================================
#Full matching function:

def manual_extract_one(nquery, a_num, a_st, cquery, nchoices, addnums,addsts, cchoices):
    '''    
    Inputs: 
    nquery: a string containing a business name from Optimus
    a_num: a string containing a business address number from Optimus
    a_st: a string containing a business address remainder from Optimus
    cquery: a string containing a business' city from Optimus
    nchoices: a column of a dataframe containg strings of business names from Gov data
    addnums: a column of a dataframe containg strings of address numbers from Gov data
    addsts: a column of a dataframe containg strings of address remainders from Gov data
    cchoices: a column of a dataframe containg strings of business cities from Gov data

    Outputs:
    best_index: The index of the row in the Gov dataframe that is the best match for the current row in Optimus
    best_score: the best total Match Score for the row
    BestN: The name score for the best index row for business name
    BestA: The address score for the best index row for business address
    BestC: The city score for the best index row for business city

    Description: This function applies the name and address matching functions from above to whole rows of Governmen data. It selects a single row
    of Optimus data and then iterates row by row through the Gov data, until it finds the best match
    '''
    #Initializing best score and best index variables
    best_score = 0
    best_index = -1
    bestN=''
    bestA=''
    bestC=0
    Nscore=0
    Ascore=0
    Cscore=0
    #Main loop that goes row by row
    for index, (nchoice, a_num_choice, a_st_choice, cchoice) in enumerate(zip(nchoices, addnums, addsts, cchoices)):
        # Perform fuzzy matching for Name and Addresss
        Nscore = n_fm(nquery, nchoice)
        Ascore = a_fm(a_num, a_st, a_num_choice, a_st_choice)
        #City Score
        Cscore=n_fm(cquery, cchoice)
        #If city score is greater than .9, the penalty term is set to 0
        if Cscore >.9:
            Cmod=0
        #Apply a penalty term for low city score of -.1
        else:
            Cmod= .1
        # Average the scores and subtract the City penalty
        avg_score = ((0.5*Nscore) + (0.5*Ascore))-Cmod

        # Update best match if the new average score is better
        if avg_score > best_score:
            best_score = avg_score
            best_index = index
            bestN=Nscore
            bestA=Ascore
            bestC=Cscore
          
        # If a good match occurs, stop the loop to speed up processing time
        if best_score >= .92:
            break
  
    return best_index, best_score, bestN, bestA, bestC

In [22]:
#Function that applies manual_extract_one to every row of optimus
def find_match(row, choices):
    ''' 
    input:
    row: A single row of the Optimus dataframe
    choices: The government dataframe

    Output: A single list that contains all the outputs of the manual_extract_one. It will contain False and 0's in the event of an error

    Description: This function narrows down the potential choices of the Gov dataframe to only rows with the same first four digits of zipcode. It also handles errors and the event
    of an empty choices df
    '''
    try:
        #Filter down the potential matches by approx zip code
        choices_filtered = choices[(choices['APPROX_ZIP_GOV'] == row['APPROX_ZIP_OPT'])]

        if choices_filtered.empty:
            return [False, 0, 0, 0,0]
        
        # Call the manual_extract_one function for fuzzy matching scores
        index, total_score, namescore, addscore, cscore = manual_extract_one(row['Cleaned_Name'], 
                                                                             row['O_Add_#'], row['O_Add_Name'], 
                                                                             row['CITY_OPT'], choices_filtered['Adj_BN'], 
                                                                             choices_filtered['G_Add_#'], choices_filtered['G_Add_Name'], choices_filtered['City'])
        #Handle the case where no match is found
        if index is None:
            return [None, 0, 0, 0,0]
        # Return results when match is found
        else:
            original_index = choices_filtered.index[index]
         
            return [original_index, total_score, namescore, addscore, cscore]
    #Handle all errors and empty rows
    except Exception as e:
        # Return default values 
        print(f"Error in find_match: {e}")
        return [False, 0, 0, 0,0]


In [23]:
#================================================================================================
#Main Function
def main(DF, Gov):
    ''' 
    input: 
    DF: This is the optimus df
    Gov: This is the Gov df

    Outputs:
    RDF: A formatted df with the matches appended

    Description: This function runs find match and handles loading bars, and reformats the results intoa df
    '''
    results = []  # List to collect result dictionaries
    
    # Iterate through rows of Optimus data
    for _, row in tqdm(DF.iterrows(), total=DF.shape[0], desc='Performing Fuzzy Matching'): 
        # Get the best match index and score from find_match
        MATCH = find_match(row, Gov)
    
        # Skip if no match found
        if not MATCH:
            continue

        # Unpack the match results into individual variables
        match_index, score, namescore, addscore, cscore = MATCH[0], MATCH[1], MATCH[2], MATCH[3], MATCH[4]
        
        # Check if a valid match was found
        if match_index is False:
            # Handle case when no valid match is found by creating a blank row
            blank = pd.DataFrame(np.nan, columns=Gov.columns, index=[0])
            mrow = blank.iloc[0]
        else:
            # Get the matched row from Gov data using the index
            mrow = Gov.iloc[match_index]
        
        # Create a result dictionary with the selected data from both Optimus and Gov
        result = {
            "OPTI_Name": row['PRIMARYINSUREDNAME'],
            "Name Score": namescore,
            "Address Score": addscore,
            "City Score": cscore, 
            "Match Score": score,
            "Opti_Address": row['STREETADDRESS_OPT'], 
            "Opti_City": row['CITY_OPT'],  
            "Opti_Zip": row['ZIP_OPT']
        }

        # Add matched row data from Gov to result
        result.update(mrow.to_dict())  
        results.append(result)
    #Convert to a pandas dataframe
    RDF = pd.DataFrame(results)
    #Select custom columns
    RDF=RDF[['OPTI_Name','Business Name', 'Name Score',
              'Opti_Address', 'Address 1', 'Address 2', 'Address Score', 
              'Opti_City','City', "City Score",
              'Opti_Zip', 'Zip', 'Match Score',
              'State', 'Geocoded Location', 'Trade name', 'Adj_BN', 'Adj_Add', 'G_Add_#', 'G_Add_Name' ]]
    RDF=pd.DataFrame(RDF)
    return RDF

In [24]:
df=ODF.sample(5000, random_state=178)

#11 is a good seed
R=main(df,GDF)


#Resturcturing R


RQ = R[(R['Match Score']>0.74) ] 
RQ.sort_values(by='Match Score', ascending=True, inplace=True)
RQ['Zip']=RQ['Zip'].astype(int)
print(f"{(len(RQ)/len(R))*100}% Match Rate")
RQ.head(45)





Performing Fuzzy Matching: 100%|██████████| 5000/5000 [23:52<00:00,  3.49it/s]  


9.379999999999999% Match Rate


,OPTI_Name,Business Name,Name Score,Opti_Address,Address 1,Address 2,Address Score,Opti_City,City,City Score,Opti_Zip,Zip,Match Score,State,Geocoded Location,Trade name,Adj_BN,Adj_Add,G_Add_#,G_Add_Name
1793,MD ENTERPRISES LLC,"R.k.h. Enterprises, Inc.",0.831793,207 W PENN AVE,209 W Penn Ave,,0.650000,cleona,cleona,1.000000,17042,17042,0.740897,PA,(-76.479451228 40.336690608),No Trade Name,rkh enterprises,209 w penn ave,209,w penn ave
4860,SPECIALTY NAMEPLATE MANUFACTURING INC,"Specialty Cruises, Inc.",0.682402,85 GARFIELD ST,85 Garfield Street,,1.000000,w middlesex,west middlesex,0.880000,16159,16159,0.741201,PA,(-80.458089998 41.172161992),No Trade Name,specialty cruises,85 garfield st,85,garfield st
3299,JUSTWORKS EMPLOYMENT GROUP LLC,"Asher Orion Group, Llc",0.683314,461 COCHRAN RD 146,461 Cochran Road,Suite 137,0.800000,pittsburgh,pittsburgh,1.000000,15228,15228,0.741657,PA,(-80.052173035 40.383105008),No Trade Name,asher orion group,461 cochran rd # 137,461,cochran rd # 137
1084,SANSOM STREET INC,"Sansom Street, Inc.",1.000000,651657 W GLENWOOD AVE,649 W Glenwood,,0.483333,philadelphia,philadelphia,1.000000,19140,19140,0.741667,PA,(-75.141422897 40.001506445),No Trade Name,sansom street,649 w glenwood,649,w glenwood
999,MACHELL MANAGEMENT INC,"Machell Management, Inc.",1.000000,1 STERLING AVE,One Sterling Avenue,,0.483333,dallas,dallas,1.000000,18612,18612,0.741667,PA,(-75.969905918 41.332142016),No Trade Name,machell management,one #rling ave,,one #rling ave
2685,TELLIGEN INC,Light in Lens Llc,0.483550,100 MAIN ST,100 Main Street,,1.000000,newtown square,newtown square,1.000000,19073,19073,0.741775,PA,(-75.41373804 39.983557998),No Trade Name,light in lens,100 main st,100,main st
45,WHEATLAND FEDERAL CREDIT UNION,Hempfield Foundation,0.484286,200 CHURCH ST,200 Church St,,1.000000,landisville,landisville,1.000000,17538,17538,0.742143,PA,(-76.410625026 40.091883997),No Trade Name,hempfield foundation,200 church st,200,church st
3969,PACE RESOURCES INC,"Basco Associates, P.c.",0.484848,445 W PHILADELPHIA ST,445 W Philadelphia St,,1.000000,york,york,1.000000,17404,17401,0.742424,PA,(-76.739691001 39.960831013),No Trade Name,basco associates pc,445 w philadelphia st,445,w philadelphia st
618,NANCYS CANDY CORNER INC,"Croy Cabinetry, Inc.",0.484848,8301 TORRESDALE AVE,8301 Torresdale Ave,,1.000000,philadelphia,philadelphia,1.000000,19136,19136,0.742424,PA,(-75.01653197 40.037981989),No Trade Name,croy cabinetry,8301 torresdale ave,8301,torresdale ave
4886,FNB CORP,Semco Corporation,0.485000,681 CLAIRTON BLVD,681 Clairton Blvd,,1.000000,pittsburgh,pittsburgh,1.000000,15236,15236,0.742500,PA,(-79.944403972 40.329299015),No Trade Name,semco corporation,681 clairton blvd,681,clairton blvd


In [25]:
Anum=100
Aname='commerce dr'
Bnum=963
Bname='winterset rd'
a_fm(Anum, Aname, Bnum, Bname)

0.08478260869565218

In [26]:
def dc_search(df, search_value, column_name):
    # Filter the DataFrame to only include rows where the specified column contains the search value
    filtered_df = df[df[column_name] == search_value]
    return(filtered_df.head())
# dc_search(GDF,'Polymath Park Resort Inc.','Business Name' )